### Lập trình mạng nơ ron hồi tiếp từ đầu
- Trong phần này, ta lập trình từ đầu mô hình ngôn ngữ được giới thiệu trong chương trước. Mô hình này dựa trên mạng nơ ron hồi tiếp ở cấp độ ký tự được huấn luyện trên tiểu thuyết The Time Machine. Ta bắt đầu với việc đọc tập dữ liệu.

In [365]:
from d2l import torch as d2l
import math
import torch
from torch import nn
from torch.nn import functional as F

batch_size, num_steps = 32, 35
data = d2l.TimeMachine(batch_size, num_steps)

In [366]:
print(data.vocab.token_freqs)

[(' ', 32775), ('e', 17838), ('t', 13515), ('a', 11704), ('i', 10138), ('n', 9917), ('o', 9758), ('s', 8486), ('h', 8257), ('r', 7674), ('d', 6337), ('l', 6146), ('m', 4043), ('u', 3805), ('c', 3424), ('f', 3354), ('w', 3225), ('g', 3075), ('y', 2679), ('p', 2427), ('b', 1897), ('v', 1295), ('k', 1087), ('x', 236), ('z', 144), ('j', 97), ('q', 95)]


In [367]:
data_iter = data.get_dataloader(train = True)


In [368]:
# In ra một minibatch
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 5,  0, 21,  ...,  0, 15, 16],
        [ 0,  9, 10,  ...,  2, 19,  2],
        [ 2, 23,  6,  ..., 13, 10, 21],
        ...,
        [ 0, 21, 19,  ...,  2, 10,  5],
        [ 6, 19,  2,  ..., 19, 16,  4],
        [16, 13, 13,  ..., 21, 16,  0]]) tensor([[ 0, 21, 16,  ..., 15, 16, 21],
        [ 9, 10, 20,  ..., 19,  2,  5],
        [23,  6, 13,  ..., 10, 21, 21],
        ...,
        [21, 19,  2,  ..., 10,  5,  0],
        [19,  2,  3,  ..., 16,  4,  6],
        [13, 13, 16,  ..., 16,  0,  4]])


In [369]:
vocab = data.vocab
print(vocab.to_tokens(y[0]))
str = ""
for chr in vocab.to_tokens(y[0]):
    str += chr
print(str)

[' ', 't', 'o', ' ', 'h', 'e', 'r', 'e', ' ', 's', 'u', 'r', 'e', 'l', 'y', ' ', 't', 'h', 'e', ' ', 'm', 'e', 'r', 'c', 'u', 'r', 'y', ' ', 'd', 'i', 'd', ' ', 'n', 'o', 't']
 to here surely the mercury did not


In [370]:
###############
train_iter, vocab = data.get_dataloader(train = True), data.vocab

#### 1. Biểu diễn one hot

In [371]:
# Biểu diễn one-hot trong pytorch
F.one_hot(input = torch.tensor([0, 1, 2, 3]), num_classes=4)

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

Kích thước minibatch mà ta lấy mẫu ở mỗi lần là __(Kích thước batch, bước thời gian)__. Hàm one_hot biển đổi một minibatch như vậy thành một tensor 3 chiều với kích thước chiều cuối cùng bằng bộ từ vựng. Chúng ta thường xuyên chuyển vị đầu vào để có đầu ra với kích thước __(bước thời gian, kích thước batch, kích thước bộ từ vựng)__ phù hợp hơn để đưa vào mô hình chuỗi.

In [372]:
print(X.shape)
print(F.one_hot(input = X.T, num_classes= len(vocab)).shape)

torch.Size([32, 35])
torch.Size([35, 32, 28])


#### 2. Khởi tạo tham số mô hình
- Tiếp theo ta khởi tạo các tham số cho mô hình RNN. Số nút ẩn num_hiddens là tham số có thể điều chỉnh.

In [373]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    # Hidden layer parameters
    W_xh = torch.rand(size = (num_inputs, num_hiddens), device = device, dtype=torch.float32)
    W_hh = torch.rand(size = (num_hiddens, num_hiddens), device = device, dtype=torch.float32)
    b_h = torch.zeros(size = (1, num_hiddens), device = device, dtype=torch.float32)

    # Output layer parameters
    W_hq = torch.rand(size = (num_hiddens, num_outputs), device = device, dtype=torch.float32)
    b_q = torch.zeros(size = (1, num_outputs), device = device, dtype=torch.float32)

    params = [W_xh, W_hh, b_h, W_hq, b_q]

    for param in params:
        param.requires_grad_(True)
        param = param.to(device)

    return params

#### 3. Mô hình RNN

- Đầu tiên ta khởi tạo trạng thái ẩn bằng hàm init_rnn_state. Hàm này trả về tuple gồm một ndarray chứa giá trị 0 và có kích thước là (kích thước batch, số nút ẩn). 
- Trả về tuple giúp ta dễ dàng xử lý các tính huống khi trạng thái ẩn có nhiều biến (Ví dụ ta cần khởi tạp nhiều tầng được kết hợp trong rnn).

In [374]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros(
        size = (batch_size, num_hiddens), device = device, dtype=torch.float32
    ), )

Hàm rnn sau định nghĩa cách tính toán trạng thái ẩn và đầu ra tại một bước thời gian. Hàm kích hoạt ở đây là tanh. Giá trị trung bình của hàm tanh là 0, khi các phần tử được phân bố đều trên trục số thực.

In [375]:
def rnn(inputs, state, params):
    # Input shape: (num_steps, batch_size, vocab_size)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H = state[0]

    outputs = []
    for X in inputs:
        # print(H.shape, b_h.shape)
        H = F.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)

    outputs = torch.concatenate(outputs, axis = 0), (H, )
    return outputs


In [376]:
class RNNScratch(d2l.HyperParameters):
    def __init__(self, vocab_size, num_hiddens, get_params,
                 init_state, forward_fn, device):
        self.save_hyperparameters()
        self.params = get_params(vocab_size, num_hiddens, device)
    
    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size)
        X = X.to(dtype = torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [377]:
num_hiddens = 512
device = 'cuda'

model = RNNScratch(len(vocab), num_hiddens, get_params, init_rnn_state, rnn, device)

state = model.begin_state(X.shape[0], device)
X = X.to(device)
Y, new_state = model(X, state)

print(Y.shape, new_state[0].shape)
# y.shape = 35 x 32 với 35 là số step, 32 là số batch
# Sau các step thì concat tất cả lại với nhau

torch.Size([1120, 28]) torch.Size([32, 512])


Có thể thấy kích thước đầu ra là (số bước x kích thước batch, kích thước bộ từ vựng), trong khi kích thước trạng thái ẩn vẫn giữ nguyên là (kích thước batch, số nút ẩn)

#### 4. Dự đoán
- Trước tiên ta tạo hàm dự đoán thường xuyên được dùng để kiểm tra trong quá trình huấn luyện. Hàm này dự đoán num_prediects ký tự tiếp theo dựa trên prefix (một chuỗi chứa một vài ký tự). Ở các ký tự đầu tiên trong chuỗi, ta chỉ cập nhật trạng thái ẩn rồi sau đó mới bắt đầu tạo ra các ký tự mới.

In [378]:
prefix = "aimes new"
vocab[prefix[0]]
# vocab.idx_to_token[2]

2

In [379]:
def predict(prefix, num_predicts, model, vocab, device):
    state = model.begin_state(batch_size = 1, device = device)
    outputs = [vocab[prefix[0]]] # chỉ số của chữ cái đầu tiên trong prefix
    print(outputs, vocab[prefix[0]])
    def get_input():
        return torch.tensor(data = [outputs[-1]], device = device).reshape(1, 1)
    
    for y in prefix[1:]: # Tạo những chữ cái của prefix trong output và 
        _, state = model(get_input(), state)
        outputs.append(vocab[y])

    for i in range(num_predicts):
        y, state = model(get_input(), state)
        outputs.append(y.argmax(axis = 1).reshape(1))
    
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [380]:
predict('time traveller ', 10, model, vocab, device)

[21] 21


'time traveller cccccccccc'